In [ ]:
import requests
import time 
import datetime
import csv
import numpy as np
import pandas as pd

In [ ]:
#Вставьте свой токен
access_token = ''
owner_id = '-215426617' #1tsprint vk id

### Функция получения ответа API vk.com в виде Json

In [ ]:
def getjson(url, data = None):
    response = requests.get(url, params = data)
    response.raise_for_status()
    print(response.url, '\n')
    return response.json()

### Функция получения всех записей на стене сообщества и их общего количества, метод wall.get

In [ ]:
def get_all_posts(access_token, owner_id, count = 100, offset=0):
    all_posts = []
     
    while True:
        time.sleep(1)
        wall = getjson("https://api.vk.com/method/wall.get", {
            'owner_id' : owner_id, 
            'count' : count,
            'access_token' : access_token,
            'offset' : offset,
            'v' : '5.92'
            })
         
        count_posts = wall['response']['count']
        posts = wall['response']['items']
         
        all_posts.extend(posts)
         
        if len(all_posts) >= count_posts:
            break
        else:
            offset += 100
            
    return all_posts, count_posts

### Функция обработки ответа API vk.com (полученных методом wall.get)

In [ ]:
def make_posts(all_posts):
    filtered_data = []
    for post in all_posts:
         
        try:
            id = post['id']
        except:
            id = 0
             
        try:
            owner_id = str(post['owner_id'])[1:]
        except:
            owner_id = ''
             
        try:
            link = 'https://vk.com/wall-{owner_id}_{id}'.format(owner_id = owner_id, id = id)
        except:
            link = ''
             
        try:
            date = datetime.datetime.fromtimestamp(int(post['date'])).strftime('%d-%m-%Y')
        except:
            date = ''
         
        try:
            likes = post['likes']['count']
        except:
            likes = 0
             
        try:
            reposts = post['reposts']['count']
        except:
            reposts = 0
             
        try:
            comments = post['comments']['count']
        except:
            comments = 0
             
        try:
            text = post['text']
        except:
            text = ''
             
        filtered_post = {
                'id' : id,
                'date' : date,
                'likes' : likes,
                'reposts' : reposts,
                'comments' : comments,
                'text' : str(text),
                'link' : link,
                'group_id' : owner_id,
                }
         
        filtered_data.append(filtered_post)
     
    return filtered_data

### Извлечение данных из ответа API vk.com  (полученных методом wall.get)

In [ ]:
all_posts, count_posts = get_all_posts (access_token, owner_id)
ref_posts = make_posts(all_posts)
count_reposts = sum([dic['reposts'] for dic in ref_posts])
ref_posts

### Получение количества подписчиков, , метод groups.getById

In [ ]:
wall = requests.get("https://api.vk.com/method/groups.getById", {
        'group_ids' : owner_id[1:],
        'grour_id' : owner_id[1:],
        'access_token' : access_token,
        'fields' : 'members_count',
        'v' : '5.131'
        })
wall.raise_for_status()
count_users = wall.json()['response'][0]['members_count']

### Запись данных заданий 1-3 в файл task_3.2.csv

In [ ]:
answer = list()
answer.append({'Показатель': 'Количество подписчиков', 'Значение':count_users})
answer.append({'Показатель': 'Количество постов', 'Значение':count_posts})
answer.append({'Показатель': 'Количество репостов', 'Значение':count_reposts})                   

with open('task_3.2.csv', 'w', newline='') as f:
    fieldnames = ['Показатель', 'Значение']
    writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    writer.writerows(answer)

### Конверсия данных, полученных со стены vk.com, в pandas.DataFrame

In [ ]:
df = pd.DataFrame(ref_posts)

### Решение задания п. 4

In [ ]:
from datetime import datetime
import dateutil.relativedelta

df4 = df[['text', 'likes', 'date']]
df4['date'] = df4['date'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y').date())

# определяем дату 3 месяца назад и фильтруем данные
init_date = (datetime.today() - dateutil.relativedelta.relativedelta(months=3)).date()
df4 = df4[df4['date'] > init_date] 

# сортировка по количеству лайков и запись результата в файл question_4.csv
df4.sort_values(by='likes', ascending=False).head(10).to_csv('question_4.csv', sep=';', encoding='utf-8', index=False)

### Решение задания п. 5

In [ ]:
df5 = df[['id', 'text', 'comments', 'date']]
# запись результата в файл question_5.csv
df5.sort_values(by='comments', ascending=False).head(10).to_csv('question_5.csv', sep=';', encoding='utf-8', index=False)

### Решение задания п. 6

In [ ]:
# список id поста и количества комментариев к нему
df6 = df[['id', 'comments']]
comm_lst = df6.values.tolist()

#### Получение данных о комментариях, метод wall.getComments

In [ ]:
comments = list()
offset = 0

for line in comm_lst:
    response = requests.get('https://api.vk.com/method/wall.getComments',
                            params={
                                'access_token': access_token,
                                'v': '5.131',
                                'owner_id': owner_id,
                                'count': line[1],
                                'offset': offset,
                                'post_id': line[0]
                                }
                            )
    data = response.json()['response']['items']
    comments.extend(data)
comments[:3]

In [ ]:
ref_comments = list()
for line in comments:
    _ = dict()
    _['post_id'] = line['post_id']
    _['id'] = line['id']
    _['date'] = line['date']
    _['text'] = line['text']
    if line['from_id'] < 0:
        line['from_id'] = -(line['from_id'])
    else:
        line['from_id']
    _['from_id'] = line['from_id']
    ref_comments.append(_)

#### Получение данных об авторах комментариев, метод users.get

In [ ]:
_ = pd.DataFrame(ref_comments)
df6 = _[['from_id']]
users_lst = df6.values.tolist()

users = []
for line in users_lst:
    response = requests.get('https://api.vk.com/method/users.get',
                            params={
                                'access_token': access_token,
                                'user_ids': line[0],
                                'v': '5.92'
                                }
                            )
    data = response.json()['response']
    users.extend(data)

    
ref_users = []

for line in users:
    _ = dict()
    _['id'] = line['id']
    _['full_name'] = line['first_name'] + ' ' + line['last_name']
    ref_users.append(_)

#### Сведение всех данных и запись в файл question_6.csv

In [ ]:
posts_df = df[['id', 'date', 'text']]

comments_df = pd.DataFrame(ref_comments)
comments_df['date'] = comments_df['date'].apply(lambda x: datetime.fromtimestamp(x / 1000).date())

users_df = pd.DataFrame(ref_users)

In [ ]:
_ = comments_df.merge(users_df, left_on='from_id', right_on='id', how='inner')
_.drop_duplicates(keep='first', inplace=True)
_ = posts_df.merge(_, left_on='id', right_on='post_id', how='left')
_.fillna(0, inplace= True)
_=_.astype({'id_x':np.int64})
_=_.drop(columns=['post_id', 'from_id', 'id_y'])
_=_.astype({'id':np.str, 'id_x':np.str, 'text_y':np.str})
_=_.rename(columns={'id': 'id поста', 'date_x': 'дата поста', 'text_x': 'текст поста', 'id_x': 'id комментария', \
                    'date_y': 'дата комментария', 'text_y': 'текст комментария', 'full_name': 'автор комментария'})

# файл question_6.csv с полными данными парсинга постов группы
_.to_csv('question_6.csv', sep=';', encoding='utf-8', index=False)

_! по условию задания необходимо отфильтровать посты группы по упоминанию в комментариях о Data Engineering, однако таких комментариев не обнаружено, поэтому в файл question_6_filt.csv я записываю данные о постах, упоминающихо Data Engineering_

In [ ]:
_['col1'] = _['текст поста'].str.lower()
_['col2'] = _['col1'].str.contains('дата инженер|data eng|инженер данных')
_ = _[_['col2'] == True]
_=_.drop(columns=['col2', 'col1'])

_.to_csv('question_6_filt.csv', sep=';', encoding='utf-8', index=False)